Пример Spark Legacy Streaming
=============================

Пример с использованием Spark Streaming. Это устаревшая технология.

## Импорты

In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import $ivy.`org.apache.spark::spark-streaming:2.4.0`
import $ivy.`org.apache.spark::spark-streaming-kafka-0-10:2.4.0`

In [ ]:
import org.apache.kafka.clients.consumer.ConsumerRecord
import org.apache.kafka.common.serialization.StringDeserializer
import org.apache.spark.streaming.kafka010._
import org.apache.spark.streaming.kafka010.LocationStrategies.PreferConsistent
import org.apache.spark.streaming.kafka010.ConsumerStrategies.Subscribe

import org.apache.spark._
import org.apache.spark.streaming._
import org.apache.spark.streaming.StreamingContext._ // not necessary since Spark 1.3

## Streaming

In [ ]:
// Create a local StreamingContext with two working thread and batch interval of 1 second.
// The master requires 2 cores to prevent a starvation scenario.

val conf = new SparkConf()
  .setMaster("local[*]")
  .setAppName("NetworkWordCount")
val ssc = new StreamingContext(conf, Seconds(1))

val kafkaParams = Map[String, Object](
  // Использование Docker DNS позволяет обращаться к контейнерам по именам внутри одной
  // Docker-сети. Поэтому в `bootstrap.servers` прописано имя контейнера:
  "bootstrap.servers" -> "kafka:9092",
  "key.deserializer" -> classOf[StringDeserializer],
  "value.deserializer" -> classOf[StringDeserializer],
  "group.id" -> "use_a_separate_group_id_for_each_stream",
  "auto.offset.reset" -> "latest",
  "enable.auto.commit" -> (false: java.lang.Boolean)
)

val topics = Array("rawdata.range")
val stream = KafkaUtils.createDirectStream[String, String](
  ssc,
  PreferConsistent,
  Subscribe[String, String](topics, kafkaParams)
)

val query = stream.map(record => (record.key, record.value))
//  .writeStream
//  .outputMode("complete")
//  .format("console")
//  .start()

In [ ]:
query.print()

In [ ]:
ssc.start()             // Start the computation
ssc.awaitTermination()  // Wait for the computation to terminate